In [ ]:
### vnhtr
from PIL import Image
from vnhtr.vnhtr_script.tools import *

predictor = VGGTransformer("cuda")

# predictor.predict([Image.open("/content/out_sample_2.jpg")])

predictor.predict([Image.open("/home/linh/projects/personal/DATA/test_self.jpg")])
# tra_predictor.predict([Image.open("/content/out_sample_2.jpg")])

/home/linh/.virtualenvs/bert/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/linh/.virtualenvs/bert/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


VGG Transformer with Rethinking Head encoder weights /tmp/vta_encoder.pt exsits. Ignore download!
VGG Transformer with Rethinking Head decoder weights /tmp/vta_decoder.pt exsits. Ignore download!
Warming up ...


['- vă mánh Kinh ']

In [9]:
from datasets import load_dataset
iam_line = load_dataset("Teklia/IAM-line")["train"]
raw = load_dataset("vklinhhh/vnhwt_opt_2")["train"]

In [2]:
iam_line

Dataset({
    features: ['image', 'text'],
    num_rows: 6482
})

In [3]:
raw

Dataset({
    features: ['image', 'label', 'full_character', 'type', 'base_character', 'diacritic_type'],
    num_rows: 355567
})

In [10]:
# --- Actual Solution Starts Here ---

# 1. Extract the 'text' column from iam_line and 'label' column from raw
iam_texts_list = iam_line['text']
raw_labels_list = raw['label']

# 2. Convert these lists to sets to get unique values and for efficient operations
iam_texts_set = set(iam_texts_list)
raw_labels_set = set(raw_labels_list)

print(f"\nNumber of unique texts in iam_line: {len(iam_texts_set)}")
print(f"Number of unique labels in raw: {len(raw_labels_set)}")

# 3. Check if iam_texts_set is a subset of raw_labels_set
is_subset = iam_texts_set.issubset(raw_labels_set)

print(f"\nIs iam_line['text'] a subset of raw['label']? {is_subset}")

# Optional: If it's not a subset, find out which texts are missing
if not is_subset:
    missing_texts = iam_texts_set - raw_labels_set # Set difference
    print(f"Number of texts from iam_line not found in raw['label']: {len(missing_texts)}")
    if len(missing_texts) < 20: # Print a few examples if the list isn't too long
        print("Examples of missing texts:")
        for i, text in enumerate(missing_texts):
            if i >= 5: # Print at most 5 examples
                print("... and more.")
                break
            print(f"- '{text}'")
    else:
        print("Many texts are missing. Consider saving them to a file if needed.")
else:
    print("All texts from iam_line are present in raw['label'].")


Number of unique texts in iam_line: 976
Number of unique labels in raw: 172331

Is iam_line['text'] a subset of raw['label']? False
Number of texts from iam_line not found in raw['label']: 762
Many texts are missing. Consider saving them to a file if needed.


In [1]:
import pandas as pd
char_mapping_df = pd.read_csv('/home/linh/projects/personal/DATA/vn_character.csv')

char_info = {}
for _, row in char_mapping_df.iterrows():
    char_info[row['label_name']] = {
        'base_character': row['base_character'],
        'type': row['label_type'],
        'diacritic': row['diacritic']
    }

In [10]:
df_final = pd.DataFrame(iam_line)
df_final.rename(columns={'text': 'label'}, inplace=True)
df_final

,image,label
0,<PIL.JpegImagePlugin.JpegImageFile image mode=...,put down a resolution on the subject
1,<PIL.JpegImagePlugin.JpegImageFile image mode=...,and he is to be backed by Mr. Will
2,<PIL.JpegImagePlugin.JpegImageFile image mode=...,nominating any more Labour life Peers
3,<PIL.JpegImagePlugin.JpegImageFile image mode=...,M Ps tomorrow. Mr. Michael Foot has
4,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"Griffiths, M P for Manchester Exchange ."
...,...,...
6477,<PIL.JpegImagePlugin.JpegImageFile image mode=...,beside # Catherine . She had the stories open
6478,<PIL.JpegImagePlugin.JpegImageFile image mode=...,In the train going back to London I sat
6479,<PIL.JpegImagePlugin.JpegImageFile image mode=...,I wish I went to that school . Did you notice
6480,<PIL.JpegImagePlugin.JpegImageFile image mode=...,that girl who said hullo to him in the garden ?


In [11]:
df_final['full_character'] = df_final['label'].apply(lambda x: list(x))
df_final['base_character'] = df_final['full_character'].apply(lambda x: [char_info[char]['base_character'] if char in char_info else char for char in x])
df_final['type'] = df_final['full_character'].apply(lambda x: [char_info[char]['type'] if char in char_info else 'need_remove' for char in x])
df_final['diacritic_type'] = df_final['full_character'].apply(lambda x: [char_info[char]['diacritic'] if char in char_info else 'no_diacritic' for char in x])

In [7]:
df_final

,image,label,full_character,base_character,type,diacritic_type
0,<PIL.JpegImagePlugin.JpegImageFile image mode=...,It was a splendid interpretation of the,"[I, t, , w, a, s, , a, , s, p, l, e, n, d, ...","[I, t, , w, a, s, , a, , s, p, l, e, n, d, ...","[vowel1, consonant, space, consonant, vowel1, ...","[no_diacritic, no_diacritic, no_diacritic, no_..."
1,<PIL.JpegImagePlugin.JpegImageFile image mode=...,sympathetic C O . Paul Daneman gave another,"[s, y, m, p, a, t, h, e, t, i, c, , C, , O, ...","[s, y, m, p, a, t, h, e, t, i, c, , C, , O, ...","[consonant, vowel1, consonant, consonant, vowe...","[no_diacritic, no_diacritic, no_diacritic, no_..."
2,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"part . The rest of the cast were well chosen ,","[p, a, r, t, , ., , T, h, e, , r, e, s, t, ...","[p, a, r, t, , ., , T, h, e, , r, e, s, t, ...","[consonant, vowel1, consonant, consonant, spac...","[no_diacritic, no_diacritic, no_diacritic, no_..."
3,<PIL.JpegImagePlugin.JpegImageFile image mode=...,with James Maxwell making a fine job of the,"[w, i, t, h, , J, a, m, e, s, , M, a, x, w, ...","[w, i, t, h, , J, a, m, e, s, , M, a, x, w, ...","[consonant, vowel1, consonant, consonant, spac...","[no_diacritic, no_diacritic, no_diacritic, no_..."
4,<PIL.JpegImagePlugin.JpegImageFile image mode=...,""" The Little Key . ""","["", , T, h, e, , L, i, t, t, l, e, , K, e, ...","["", , T, h, e, , L, i, t, t, l, e, , K, e, ...","[need_remove, space, consonant, consonant, vow...","[no_diacritic, no_diacritic, no_diacritic, no_..."
...,...,...,...,...,...,...
971,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"room , which proved the elasticity of medical ...","[r, o, o, m, , ,, , w, h, i, c, h, , p, r, ...","[r, o, o, m, , ,, , w, h, i, c, h, , p, r, ...","[consonant, vowel1, vowel1, consonant, space, ...","[no_diacritic, no_diacritic, no_diacritic, no_..."
972,<PIL.JpegImagePlugin.JpegImageFile image mode=...,Diana was trailing up the gravelled drive to t...,"[D, i, a, n, a, , w, a, s, , t, r, a, i, l, ...","[D, i, a, n, a, , w, a, s, , t, r, a, i, l, ...","[consonant, vowel1, vowel1, consonant, vowel1,...","[no_diacritic, no_diacritic, no_diacritic, no_..."
973,<PIL.JpegImagePlugin.JpegImageFile image mode=...,to be met at the gates .,"[t, o, , b, e, , m, e, t, , a, t, , t, h, ...","[t, o, , b, e, , m, e, t, , a, t, , t, h, ...","[consonant, vowel1, space, consonant, vowel1, ...","[no_diacritic, no_diacritic, no_diacritic, no_..."
974,<PIL.JpegImagePlugin.JpegImageFile image mode=...,alone . Of course one couldn't say for certain...,"[a, l, o, n, e, , ., , O, f, , c, o, u, r, ...","[a, l, o, n, e, , ., , O, f, , c, o, u, r, ...","[vowel1, consonant, vowel1, consonant, vowel1,...","[no_diacritic, no_diacritic, no_diacritic, no_..."


In [12]:
filtered_df = df_final[~df_final['type'].apply(lambda x: 'need_remove' in x)]

# Print the original and new dataframe sizes to confirm filtering worked
print(f"Original DataFrame size: {len(df_final)}")
print(f"Filtered DataFrame size: {len(filtered_df)}")

Original DataFrame size: 6482
Filtered DataFrame size: 6056
